# Условие

Для данной выборки $Y$ найти такие $C_1,C_2,C_3$, что метрики
$$
MSE = \frac{1}{N}\sum_{i=1}^N (y_i - C_1)^2
$$
$$
MAE = \frac{1}{N}\sum_{i=1}^N |y_i - C_2|
$$
$$
MAPE = \frac{1}{N}\sum_{i=1}^N \left|\frac{y_i - C_3}{y_i}\right|
$$
будут минимальны.

# Решение

Для всех метрик найдём производную относительно $C$, приравняем к нулю, чтобы потом найти экстремумы, и выведем $C$.

## 1. MSE

$$
MSE_{C_1}' = \frac{1}{N}\sum_{i=1}^N 2(y_i - C_1)(-1) = \frac{2}{N}\sum_{i=1}^{N} (C_1 - y_i) = 2C_1 - \frac{2}{N}\sum_{i=1}^{N}y_i=0
$$
Если что, множитель (-1) появился, так как берем производную сложной функции $(y_i - C_1)^2$, где $C_1$ со знаком минус.
$$
C_1 = \frac{\sum_{i=1}^{N}y_i}{N}
$$

## 2. MAE

Для начала перепишем $MAE$ так, чтобы работать без модуля.
$$
MAE = \frac{1}{N}\sum_{i=1}^{N}|y_i-C_2| = \frac{1}{N}\sum_{i=1}^{N}\left[
    \begin{array} \\
        y_i - C_2, & y_i-C_2\geq0 \mbox{ or } y_i\geq C_2 \\
        C_2 - y_i, & y_i-C_2<0 \mbox{ or } y_i<C_2 \\
    \end{array}
\right.
$$
$$
MAE_{C_2}' = \frac{1}{N}\sum_{i=1}^{N}\left[
    \begin{array} \\
        -1, & y_i>C_2 \\
        1, & y_i<C_2 \\
        0, & y_i=C_2 \\
    \end{array}
\right.=0
$$
Производная состоит из слагаемых -1, 1, 0, то есть будет равна 0, когда количество -1 и 1 будет одинаковым. Это значит, что $C_2=med(Y)$, то есть медиане. Медианой будет средний по положению (индексу) элемент упорядоченной выборки.

## 3. MAPE

$$
MAPE = 100\frac{1}{N}\sum_{i=1}^{N}\left|\frac{y_i-C_3}{y_i}\right|=\frac{100}{N}\sum_{i=1}^{N}\frac{|y_i-C_3|}{|y_i|}
$$
$$
MAPE_{C_3}'=\frac{100}{N}\sum_{i=1}^{N}\left(\left[
    \begin{array} \\
        -1, & y_i>C_3 \\
        1, & y_i<C_3 \\
        0, & y_i=C_3 \\
    \end{array}
\right.*\frac{1}{|y_i|}\right)=0
$$
Здесь значение явно не вывести, так что переберём все $C_3=y_i$ в упорядоченном множестве $Y$, пересчитывая для каждого значения $MAPE_{C_3}'$, что можно сделать за $O(N)$, поддерживая сумму слева от текущего $C_3$ и справа. В момент, когда производная сменит знак на положительный с отрицательного (так будет всегда, так как при начальном $C_3=y_0$ почти всегда сумма производной отрицательна), ответом будет либо текущий $y_i$, либо $y_{i-1}$.

# Код

In [23]:
def mape(Y, c):
    m = 0
    for y in Y:
        m += abs((y - c) / y)
    return m * 100 / len(Y)

n = int(input())
y = []
for i in range(n):
    y.append(int(input()))
y.sort()
c1 = sum(y) / n
c2 = y[n // 2]
c3 = y[0] - 1 #initial value of c3, when c3 < any y[i]
mape_l, mape_r = 0, sum(-1 / x for x in y)
old_i = -1
i = 0
goes_to_mape_l = 0
prev_mape = mape_r
while i < n:
    mape_l += goes_to_mape_l
    goes_to_mape_l = 1 / y[i]
    mape_r += 1 / y[i]
    while i < n - 1 and y[i] == y[i + 1]:
        i += 1
        goes_to_mape_l += 1 / y[i]
        mape_r += 1 / y[i]
    if mape_l + mape_r >= 0:
        break
    old_i = i
    i += 1
c3 = y[i] if old_i == -1 or mape(y, y[i]) < mape(y, y[old_i]) else y[old_i]
print(c1, c2, c3, sep='\n')

 5
 1
 1
 1
 1
 1


1.0
1
1
